### 1.Carregamento de bibliotecas e configuração

In [1]:
import json
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
import numpy as np

# Marcar como True, caso deseje efetuar o treinamento a partir de um modelo que ja tenha passado pelo processo de fine-tuning 
# OBS: Modelo deve estar presente no diretório "./trained_model"
load_pretrained = True

# Treina o modelo apenas com uma parcela dos registros da tabela original.
# Ex: n = -1 = treinar com todos os itens da tabela
n_samples = -1

# Verifica se há GPUs disponíveis
if torch.cuda.is_available():
    device = torch.device("cuda")
    device_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory // (1024 ** 2) # MB
    torch.cuda.empty_cache()  # Limpa cache da GPU para garantir um melhor funcionamento do modelo
else:
    device = torch.device("cpu")
    device_name = "CPU"
    total_memory = "-"

print(f"\nTreinando no dispositivo: {device_name}")
print(f"Memória total: {total_memory} MB")

2024-09-26 02:34:26.833198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 02:34:26.983865: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 02:34:27.694495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/bring/miniconda3/envs/wsl2_machine_learning_test_env/lib/
2024-09-26 02:34:27.695608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'


Treinando no dispositivo: NVIDIA GeForce RTX 3080 Ti
Memória total: 12287 MB


### Carregamento do modelo

In [2]:
# Carrega modelo e tokenizer (BERT para Question Answering)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Armazena modelo/tokenizer original para comparação posterior
model_raw = model 
tokenizer_raw = tokenizer

# Carrega modelo com fine-tuning anterior ja realizado
if load_pretrained:
    tokenizer = AutoTokenizer.from_pretrained("./trained_model")
    model = AutoModelForQuestionAnswering.from_pretrained("./trained_model")

# Move o modelo para o dispositivo (GPU ou CPU)
model.to(device)
model_raw.to(device)

/home/bring/miniconda3/envs/wsl2_machine_learning_test_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

### Carregamento de Dataset

In [3]:
# Carregar dataset
def load_dataset(path):
    try:
        with open(path, 'r', encoding="utf-8") as file:
            data = [json.loads(line) for line in file]
        return data
    except FileNotFoundError:
        raise Exception(f"Erro: O arquivo {path} não foi encontrado.")
    except json.JSONDecodeError:
        raise Exception("Erro: O arquivo contém dados inválidos.")
    

print("\nCarregando dados de treino...")
raw_data = load_dataset("trn.json")

print("\nPreparando dados de treino...")
df_raw = pd.DataFrame(raw_data)
df_raw = df_raw[['title', 'content']].replace('', np.nan).dropna() # Selecionando as colunas necessárias (Removendo a coluna 'answer', e todas as linhas com valores vazios e nulos)
print(f"\n{len(df_raw)} registros válidos encontrados...\n")

if n_samples > 0:
    df = df_raw.sample(n=n_samples, random_state=42)
else:
    df = df_raw

print(f"\n{len(df)} registros selecionados para treinamento...\n")
df


Carregando dados de treino...

Preparando dados de treino...

1390403 registros válidos encontrados...


1390403 registros selecionados para treinamento...



,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
3,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...
7,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...
12,The Prophet,"In a distant, timeless place, a mysterious pro..."
13,Rightly Dividing the Word,--This text refers to thePaperbackedition.
...,...,...
2248608,[180 Days Warranty] ZeroLemon&reg; Samsung Gal...,Features:This is the World's highest capacity ...
2248611,Teenage Mutant Ninja Turtles Donatello Wizard ...,"With a pointed bed sheet hat, robe decorated w..."
2248612,~Shave Ready~ Shaving Straight Razor 6/8&quot;...,"Inside this amazing set is a 6/8"" round point ..."
2248617,Cont Removable Paper Label,"Continuous Length Removable Paper Label 2-3/7""..."


#### Tokenização

In [4]:
# Transforma dados em um Dataset da Hugging Face
dataset = Dataset.from_pandas(df)

# Função para encontrar os índices de início e fim da resposta no contexto
def add_token_positions(examples, tokenizer):
    """
    Adiciona as posições de início e fim das respostas no contexto e tokeniza os dados.

    Args:
        examples (dict): Dicionário com os exemplos de entrada (títulos e conteúdos).
        tokenizer: Tokenizer do modelo BERT.

    Returns:
        dict: Dados tokenizados com posições de início e fim das respostas.
    """
    start_positions = []
    end_positions = []

    # Considerando que a resposta será o conteúdo inteiro
    for i in range(len(examples['content'])):
        context = examples['content'][i]
        start_idx = 0
        end_idx = len(context) - 1  # A resposta será todo o contexto

        start_positions.append(start_idx)
        end_positions.append(end_idx)

    # Tokenizar o conteúdo
    tokenized_inputs = tokenizer(
        examples['content'],
        examples['title'],
        truncation=True,
        padding="max_length",
        max_length=512
    )

    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions

    return tokenized_inputs

# Tokeniza o dataset e adiciona as posições de início e fim das respostas
tokenized_data = dataset.map(lambda x: add_token_positions(x, tokenizer), batched=True)

Map:   0%|          | 0/1390403 [00:00<?, ? examples/s]

### Treino e Teste

In [5]:
# Função de Fine-tuning com BERT
def fine_tune_model(tokenized_datasets, tokenizer, model):
    """
    Realiza o fine-tuning do modelo BERT utilizando os dados tokenizados.

    Args:
        tokenized_datasets (Dataset): Dataset tokenizado para treinamento e validação.
        tokenizer: Tokenizer do modelo BERT.
        model: Modelo BERT para fine-tuning.

    Returns:
        Trainer: Objeto Trainer após o treinamento.
    """
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=2,
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        tokenizer=tokenizer
    )

    trainer.train()
    return trainer  # Retorna o objeto trainer para avaliação posterior
   

# Remove as colunas que não são usadas pelo modelo
tokenized_data = tokenized_data.remove_columns(['content', 'title'])

# Separa dados em treino e teste
tokenized_data = tokenized_data.train_test_split(test_size=0.2)

# Fine-tune no modelo
trainer = fine_tune_model(tokenized_data, tokenizer, model)

# Avalia o modelo após o fine-tuning
eval_results = trainer.evaluate(tokenized_data['test'])
print(f"Avaliação do modelo: {eval_results}")

# Salva o modelo treinado
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")
print("Modelo treinado e tokenizer salvos em './trained_model'.")

/home/bring/miniconda3/envs/wsl2_machine_learning_test_env/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
1,1.278700,nan
2,1.031500,nan
3,0.821300,nan


Avaliação do modelo: {'eval_loss': nan, 'eval_runtime': 2082.4421, 'eval_samples_per_second': 133.536, 'eval_steps_per_second': 16.692, 'epoch': 3.0}
Modelo treinado e tokenizer salvos em './trained_model'.


### Validando eficiêcia do modelo pré e pós fine-tuning

In [6]:
def generate_responses(model, tokenizer, questions, contexts):
    """
    Gera respostas para perguntas dadas com base nos contextos fornecidos, removendo tokens especiais.

    Args:
        model: Modelo BERT treinado.
        tokenizer: Tokenizer do modelo BERT.
        questions (list): Lista de perguntas.
        contexts (list): Lista de contextos correspondentes.

    Returns:
        str: Resposta gerada pelo modelo sem os tokens especiais.
    """
    inputs = tokenizer(contexts, questions, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Pegando os índices da resposta
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores)

    # Converter tokens de volta para string
    answer_tokens = all_tokens[answer_start:answer_end + 1]

    # Remover tokens especiais como [CLS], [SEP] e [PAD]
    answer_tokens = [token for token in answer_tokens if token not in ['[CLS]', '[SEP]', '[PAD]']]

    # Juntar os tokens restantes para formar a resposta final
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    return answer

In [13]:

# Seleciona algumas perguntas aleatoriamente para validação visual
df_samples = df.sample(n=5, random_state=42)
test_questions = df_samples["title"].sort_values().to_list()
test_contexts = df_samples["content"].sort_values().to_list()

# Testar o modelo antes do fine-tuning
print("\nRespostas geradas pelo modelo pré e pós fine-tuning:\n")
for index, row in df_samples.iterrows():
    response = generate_responses(model, tokenizer, row["title"], row["content"])
    response_raw = generate_responses(model_raw, tokenizer_raw, row["title"], row["content"])
    
    print(f"Pergunta: {row['title']}")
    print(f"Resposta: (Pré fine-tuning): {response_raw}")
    print(f"Resposta: (Pós fine-tuning): {response}")
    print("\n")


Respostas geradas pelo modelo pré e pós fine-tuning:

Pergunta: I Don't Care - Learning About Respect (Values)
Resposta: (Pré fine-tuning): 
Resposta: (Pós fine-tuning): brian moses lives in the small sussex village of crowhurst with his wife anne, a loopy labrador called honey and a collection of bad - tempered chickens. he first worked as a teacher but has now been a professional children ' s poet since 1988. to date he has over 200 books published including volumes of his own poetry such as holding the hands of angels ( salt ) and behind the staffroom door ( macmillan ), anthologies such as the secret lives of teachers and aliens stole my underpants ( both macmillan ), picture books such as beetle in the bathroom and trouble at the dinosaur cafe ( both puffin ) and non - fiction titles such as titanic : lost & saved ( wayland ). over 1 million copies of brian ' s poetry books have now been sold by macmillan and in 2005 he was nominated for both the clpe award and the spoken word aw

### Playground - Utilize esse trecho para interagir com o modelo

In [8]:
# while True:
#     print("Insira uma pertunta e uma pergunta e um contexto ao modelo:")
#     question = input("Pergunta: ")
#     context = input("Contexto: ")
#     if question == "" or context == "":
#         break
#     else:
#         response = generate_responses(model, tokenizer, question, context)
#         print(f"Pergunta: {question}")
#         print(f"Resposta: {response}")
#         print("\n")
